In [1]:
# Reload modules automatically
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import importlib
from src.graph_models import create_graph_model
import numpy as np
import torch
import exp_eval_robustness
import exp_train
from torch_sparse import SparseTensor

In [8]:
data_params = dict(
    graph_model = 'CSBM',
    classes = 2,
    n = 1000,
    n_per_class_trn = 400,
    K = 0.1,
    sigma = 1,
    avg_within_class_degree = 1.58 * 2,
    avg_between_class_degree = 0.37 * 2,
    inductive_samples = 10
)

model_params = dict(
    label="MLP",
    model="MLP",
    n_filters=64,
    dropout=0.5,
    hidden_channels=32,
    use_label_propagation=False,
)

train_params = dict(
    lr=0.1,
    weight_decay=1e-3,
    patience=300,
    max_epochs=3000,
    inductive=True,
)

attack_params = dict(
    attack = "l2",
    surrogate_model_params = dict(
        label="LinearGCN",
        model="LinearGCN",
        n_filter=64,
        dropout=0.5,
    ),
    surrogate_train_params = dict(
        lr=0.1,
        weight_decay=1e-2,
        patience=300,
        max_epochs=3000,
    )
)

verbosity_params = dict(
    display_steps = 100,
    debug_lvl = "info"
)  

other_params = dict(
    device = 0,
    allow_tf32 = False,
    sacred_metrics = True
)

seed = 0

In [9]:
importlib.reload(exp_eval_robustness)
importlib.reload(exp_train)
#result = exp_train.run(data_params, model_params, train_params, verbosity_params, other_params, seed, None)
result = exp_eval_robustness.run(data_params, model_params, train_params, attack_params, verbosity_params, other_params, seed, None)

2022-09-08 20:08:24 (INFO): Starting experiment exp_eval_robustness with configuration:
2022-09-08 20:08:24 (INFO): data_params: {'graph_model': 'CSBM', 'classes': 2, 'n': 1000, 'n_per_class_trn': 400, 'K': 0.1, 'sigma': 1, 'avg_within_class_degree': 3.16, 'avg_between_class_degree': 0.74, 'inductive_samples': 10}
2022-09-08 20:08:24 (INFO): model_params: {'label': 'MLP', 'model': 'MLP', 'n_filters': 64, 'dropout': 0.5, 'hidden_channels': 32, 'use_label_propagation': False}
2022-09-08 20:08:24 (INFO): train_params: {'lr': 0.1, 'weight_decay': 0.001, 'patience': 300, 'max_epochs': 3000, 'inductive': True}
2022-09-08 20:08:24 (INFO): attack_params: {'attack': 'l2', 'surrogate_model_params': {'label': 'LinearGCN', 'model': 'LinearGCN', 'n_filter': 64, 'dropout': 0.5}, 'surrogate_train_params': {'lr': 0.1, 'weight_decay': 0.01, 'patience': 300, 'max_epochs': 3000}}
2022-09-08 20:08:24 (INFO): verbosity_params: {'display_steps': 100, 'debug_lvl': 'info'}
2022-09-08 20:08:24 (INFO): other_pa

In [6]:
importlib.reload(exp_eval_robustness)
importlib.reload(exp_train)
#result = exp_train.run(data_params, model_params, train_params, verbosity_params, other_params, seed, None)
result = exp_eval_robustness.run(data_params, model_params, train_params, attack_params, verbosity_params, other_params, seed, None)

2022-09-07 18:16:15 (INFO): Starting experiment exp_eval_robustness with configuration:
2022-09-07 18:16:15 (INFO): data_params: {'graph_model': 'CSBM', 'classes': 2, 'n': 1000, 'n_per_class_trn': 400, 'K': 5, 'sigma': 1, 'avg_within_class_degree': 3.16, 'avg_between_class_degree': 0.74, 'inductive_samples': 10}
2022-09-07 18:16:15 (INFO): model_params: {'label': 'GAT', 'model': 'GAT', 'dropout': 0.0, 'n_heads': 8, 'n_features_per_head': 8, 'dropout_neighbourhood': 0.0, 'use_label_propagation': False}
2022-09-07 18:16:15 (INFO): train_params: {'lr': 0.1, 'weight_decay': 0.001, 'patience': 300, 'max_epochs': 3000, 'inductive': True}
2022-09-07 18:16:15 (INFO): attack_params: {'attack': 'l2', 'surrogate_model_params': {'label': 'LinearGCN', 'model': 'LinearGCN', 'n_filter': 64, 'dropout': 0.5}, 'surrogate_train_params': {'lr': 0.1, 'weight_decay': 0.01, 'patience': 300, 'max_epochs': 3000}}
2022-09-07 18:16:15 (INFO): verbosity_params: {'display_steps': 100, 'debug_lvl': 'info'}
2022-09-

In [12]:
from src.data import split
from src.models import create_model
from src.train import train_inductive, train_transductive

model_params = dict(
    label="GCN",
    model="LinearGCN",
    n_filter=64,
    dropout=0.5,
    use_label_propagation=False,
)
train_params = dict(
    lr=0.1,
    weight_decay=1e-3,
    patience=300,
    max_epochs=3000,
    inductive=True,
)
_run = None
device = torch.device(f"cuda:{0}")

# Sample Graph
graph_model = create_graph_model(data_params)
X_np, A_np, y_np = graph_model.sample(data_params["n"], seed)
X = torch.tensor(X_np, dtype=torch.float32, device=device)
A = torch.tensor(A_np, dtype=torch.float32, device=device)
y = torch.tensor(y_np, device=device)
split_trn, split_val = split(y_np, data_params)

# Create Model
model_params_trn = dict(**model_params, 
                        n_features=X_np.shape[1], 
                        n_classes=data_params["classes"])
model = create_model(model_params_trn).to(device)
#logging.info(model)

# Train Model
if train_params["inductive"]:
    train = train_inductive
else:
    train = train_transductive
trn_tracker = train(model, None, X, A, y, split_trn, split_val, train_params,
                    verbosity_params, _run)

2022-09-06 14:57:49 (INFO): 
Epoch    0: loss_train: 0.70049, loss_val: 0.69125, acc_train: 0.51250, acc_val: 0.54000
2022-09-06 14:57:49 (INFO): 
Epoch  100: loss_train: 0.66957, loss_val: 0.70510, acc_train: 0.59875, acc_val: 0.55000
2022-09-06 14:57:50 (INFO): 
Epoch  200: loss_train: 0.69992, loss_val: 0.69783, acc_train: 0.57000, acc_val: 0.59000
2022-09-06 14:57:50 (INFO): 
Epoch  300: loss_train: 0.67012, loss_val: 0.70146, acc_train: 0.59500, acc_val: 0.53000
2022-09-06 14:57:51 (INFO): 
Epoch  400: loss_train: 0.67534, loss_val: 0.70552, acc_train: 0.57750, acc_val: 0.53000
2022-09-06 14:57:51 (INFO): 
Epoch  500: loss_train: 0.69036, loss_val: 0.70550, acc_train: 0.58875, acc_val: 0.53500
2022-09-06 14:57:51 (INFO): 
Epoch  600: loss_train: 0.67120, loss_val: 0.70989, acc_train: 0.60250, acc_val: 0.53000
2022-09-06 14:57:52 (INFO): 
Epoch  700: loss_train: 0.68360, loss_val: 0.70071, acc_train: 0.56875, acc_val: 0.53500
2022-09-06 14:57:52 (INFO): 
Epoch  430: loss_train: 0.6

In [16]:
type(model)

src.models.gcn.DenseGCN

In [18]:
from src.models.gcn import DenseGCN
isinstance(model, DenseGCN)

True

In [22]:
isinstance(model.activation, torch.nn.Identity)

True

In [25]:
model.layers[0][0]._linear.weight

Parameter containing:
tensor([[-0.2845, -0.1751,  0.0541,  ...,  0.3195,  0.0939,  0.0529],
        [-0.0133, -0.1708,  0.0041,  ..., -0.0677, -0.3852, -0.0863],
        [ 0.0237, -0.1400,  0.0761,  ..., -0.0926, -0.1117, -0.0352],
        ...,
        [-0.1547, -0.6180,  0.0105,  ..., -0.4443, -0.1026, -0.1262],
        [-0.0155,  0.1586,  0.4082,  ...,  0.2353, -0.1754, -0.0009],
        [-0.1066, -0.2037, -0.0348,  ..., -0.1079, -0.1091, -0.2001]],
       device='cuda:0', requires_grad=True)

In [26]:
model_params = dict(
    label="SGC",
    model="SGC",
    K=2,
    use_label_propagation=False,
)
model_params_trn = dict(**model_params, 
                        n_features=X_np.shape[1], 
                        n_classes=data_params["classes"])
sgc = create_model(model_params_trn).to(device)

In [45]:
model.layers[0][0]

DenseGraphConvolution(
  (_linear): Linear(in_features=21, out_features=64, bias=False)
)

In [39]:
sgc.parameters

<bound method Module.parameters of SGC(
  (sgc): SGConv(21, 2, K=2)
)>

In [15]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"]

KeyError: 'CUBLAS_WORKSPACE_CONFIG'

In [9]:
data_params["sigma"] = 1
seed = 1

result = exp_eval_robustness.run(data_params, model_params, train_params, attack_params, verbosity_params, other_params, seed, None)

2022-08-10 14:11:26 (INFO): Starting experiment exp_eval_robustness with configuration:
2022-08-10 14:11:26 (INFO): data_params: {'graph_model': 'CSBM', 'classes': 2, 'n': 1000, 'n_per_class_trn': 400, 'K': 0.5, 'sigma': 1, 'avg_within_class_degree': 3.0, 'avg_between_class_degree': 1.0, 'inductive_samples': 1000}
2022-08-10 14:11:26 (INFO): model_params: {'label': 'GCN', 'model': 'DenseGCN', 'n_filters': 64}
2022-08-10 14:11:26 (INFO): train_params: {'lr': 0.01, 'weight_decay': 0.001, 'patience': 300, 'max_epochs': 3000, 'inductive': True}
2022-08-10 14:11:26 (INFO): attack_params: {'attack': 'l2'}
2022-08-10 14:11:26 (INFO): verbosity_params: {'display_steps': 100, 'debug_lvl': 'info'}
2022-08-10 14:11:26 (INFO): other_params: {'device': 0, 'allow_tf32': False, 'sacred_metrics': True}
2022-08-10 14:11:26 (INFO): seed: 1
2022-08-10 14:11:26 (INFO): db_collection: None
2022-08-10 14:11:26 (INFO): Currently on gpu device cuda:0
2022-08-10 14:11:27 (INFO): 
Epoch    0: loss_train: 0.7010

In [21]:
from src.data import calc_balanced_sample

In [22]:
class_counts = [10, 100, 100]
n_samples = 60
print(calc_balanced_sample(class_counts, n_samples))

[10 25 25]


In [35]:
from src.utils import accuracy

In [7]:
logits = torch.Tensor([[0.8, 0.2], [0.1, 0.9]])
labels = torch.Tensor([0, 1])
logits.argmax(1)[1]

tensor(1)